In [10]:
import sqlite3
import json
from datetime import datetime as dt
import time

In [11]:
tf = ['2007-10','2007-11','2007-12','2008-01','2008-02',
             '2008-03','2008-04','2008-05','2008-06','2008-07',
             '2008-08','2008-09','2008-10','2008-11','2008-12',
             '2009-01','2009-02','2009-03','2009-04','2009-05',
             '2009-06','2009-07','2009-08','2009-09','2009-10',
             '2009-11','2009-12','2015-01']
sql_transaction = []

In [12]:
def format_data(data):
    data = data.replace('\n',' ').replace('\r',' ').replace('"',"'")
    return data

def create_table():
    c.execute("""CREATE TABLE IF NOT EXISTS parent_reply
    (parent_id TEXT PRIMARY KEY, comment_id TEXT UNIQUE, parent TEXT, comment TEXT, subreddit TEXT, unix INT, score INT)""")


In [13]:
def find_parent(pid):
    try:
        sql = "SELECT comment from parent_reply WHERE comment_id = '{}' LIMIT 1".format(pid)
        c.execute(sql)
        result = c.fetchone()
        if result != None:
            return result[0]
        else:
            return False
    except Exception as e:
        return False

In [14]:
def find_existing_score(pid):
    try:
        sql = "SELECT score from parent_reply WHERE parent_id = '{}' LIMIT 1".format(pid)
        c.execute(sql)
        result = c.fetchone()
        if result != None:
            return result[0]
        else:
            return False
    except Exception as e:
        return False

def find_parent(pid):
    try:
        sql = "SELECT comment FROM parent_reply WHERE comment_id = '{}' LIMIT 1".format(pid)
        c.execute(sql)
        result = c.fetchone()
        if result != None:
            return result[0]
        else: return False
    except Exception as e:
        #print(str(e))
        return False


In [15]:
def acceptable(data):
    if len(data.split(' ')) > 50 or len(data) < 2:
        return False
    elif len(data) > 1000:
        return False
    elif data == '[deleted]':
        return False
    elif data == '[removed]':
        return False
    else:
        return True

In [16]:
def sql_insert_replace_comment(comment_id,parent_id,parent_data,body,subreddit,created_utc,score):
    try:
        sql = """UPDATE parent_reply SET parent_id = ?, comment_id = ?, parent = ?, comment = ?, subreddit = ?, 
        unix = ?, score = ? WHERE parent_id =?;""".format(parentid, commentid, parent, comment, subreddit, int(time), score, parentid)
        transaction_bldr(sql)
    except Exception as e:
        pass
        #print('s0 insertion ',str(e))

def sql_insert_has_parent(commentid,parentid,parent,comment,subreddit,time,score):
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, parent, comment, subreddit, unix, score) VALUES ("{}","{}","{}","{}","{}",{},{});""".format(parentid, commentid, parent, comment, subreddit, int(time), score)
        transaction_bldr(sql)
    except Exception as e:
        pass
        #print('s0 insertion',str(e))


def sql_insert_no_parent(commentid,parentid,comment,subreddit,time,score):
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, comment, subreddit, unix, score) VALUES ("{}","{}","{}","{}",{},{});""".format(parentid, commentid, comment, subreddit, int(time), score)
        transaction_bldr(sql)
    except Exception as e:
        pass
        #print('s0 insertion',str(e))

In [17]:
def transaction_bldr(sql):
    global sql_transaction
    sql_transaction.append(sql)
    if len(sql_transaction) > 1000:
        c.execute('BEGIN TRANSACTION')
        for s in sql_transaction:
            try:
                c.execute(s)
            except Exception as e:
                #print(e)
                pass
        connection.commit()
        sql_transaction = []
                

In [18]:
if __name__ == '__main__':
    row_counter = 0
    paired_rows = 0
    connection = sqlite3.connect('db_chat.db')
    c = connection.cursor()
    create_table()
    for timeframe in tf:
        print(timeframe)
        with open('C:\\Users\\HP\\Downloads\\reddit_data\\lol\\RC_{}'.format(timeframe), buffering=1000) as f:
            for row in f:
                #if(row_counter % 10000 == 0):print(row)
                row_counter += 1
                row = json.loads(row)
                parent_id = row['parent_id']
                body = format_data(row['body'])
                created_utc = row['created_utc']
                score = row['score']
                comment_id = row['name']
                subreddit = row['subreddit']
                parent_data = find_parent (parent_id)

                if score >= 2 :
                    existing_comment_score = find_existing_score (parent_id)
                    if existing_comment_score:
                        if score > existing_comment_score :
                            if acceptable(body):
                                #print("lol1", row_counter)
                                sql_insert_replace_comment(comment_id, parent_id, parent_data, body, subreddit, created_utc, score)

                    else:
                        if acceptable(body):
                            if parent_data:
                                #print("lol2", row_counter)
                                sql_insert_has_parent(comment_id, parent_id, parent_data, body, subreddit, created_utc, score)
                                paired_rows += 1
                            else:
                                #print("lol3", row_counter)
                                sql_insert_no_parent(comment_id, parent_id, body, subreddit, created_utc, score)
                                #print('lol')
                if row_counter % 1000000 == 0:
                    print('Total Rows Read: {}, Paired Rows: {}, Time: {}'.format(row_counter, paired_rows, str(dt.now())))

2007-10
2007-11
2007-12
2008-01
Total Rows Read: 1000000, Paired Rows: 13843, Time: 2018-01-11 01:11:59.320878
2008-02
2008-03
Total Rows Read: 2000000, Paired Rows: 31510, Time: 2018-01-11 01:16:30.011515
2008-04
2008-05
Total Rows Read: 3000000, Paired Rows: 48480, Time: 2018-01-11 01:21:05.014288
2008-06
2008-07
Total Rows Read: 4000000, Paired Rows: 68791, Time: 2018-01-11 01:25:45.182771
2008-08
Total Rows Read: 5000000, Paired Rows: 90873, Time: 2018-01-11 01:30:31.595420
2008-09
2008-10
Total Rows Read: 6000000, Paired Rows: 114889, Time: 2018-01-11 01:35:15.252099
2008-11
Total Rows Read: 7000000, Paired Rows: 140268, Time: 2018-01-11 01:39:53.909129
2008-12
Total Rows Read: 8000000, Paired Rows: 166689, Time: 2018-01-11 01:43:51.803302
2009-01
Total Rows Read: 9000000, Paired Rows: 194728, Time: 2018-01-11 01:47:44.475745
2009-02
Total Rows Read: 10000000, Paired Rows: 222853, Time: 2018-01-11 01:51:17.152695
2009-03
Total Rows Read: 11000000, Paired Rows: 252019, Time: 2018-0